# Rotational camera case estimating camera rotation from homography

In this recipe, you will learn how to extract rotation from a homography transformation between two views captured by a camera undergoing only rotation motion with respect to its optical center. This is useful if, for example, you need to estimate  the rotation between two views, assuming that the translation is negligible compared to distances to scene points. That's often the case in landscape photo stitching.

In case the camera undergoes rotation only around its optical center, the homography transformation has a really simple form—it's basically a rotation matrix, but is multiplied by camera matrix parameters since homography works in image pixel space. As a first step, we factor out camera parameters from the homography matrix. After that, it must be a rotation matrix (up to scale). Since there might be noise in the homography parameters, the resulting matrix might not be a proper rotation matrix, for example, an orthogonal matrix with a determinant equal to one. That's why we construct the closest (in the Frobenius norm) rotation matrix using a singular value decomposition.

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import os

%matplotlib auto
%pylab inline

def print_image(header,name,np_arr,
                start_First=0,end_First=1,start_Second=0,end_Second=2,start_3=0,end_3=5):
    print("------  {0:-<25}    Shape{1} {2}: {3:}".format(header, np_arr.shape, name, str(np_arr.dtype)) )
    shapes = np_arr.shape #print(shapes)
    if shapes[0] < end_First:
        end_First = shapes[0]
    if shapes[1] < end_Second:
        end_Second = shapes[1]
    if len(shapes)==3:
        if shapes[2] < end_3:
            end_3 = shapes[2]
    if len(shapes)==3:
        for i in range (start_First,end_First):
            print("[", sep='',end="")
            for j in range (start_Second,end_Second):
                print(np_arr[i,j,start_3:end_3], sep=' ', end=" ")
            print(']')
    if len(shapes)==2:
        for i in range (start_First,end_First):
            print("[", end=" ")
            #print(np_arr[i,start_Second:end_Second],sep=' ',end=" ") cutoff sting by<60
            for k in range (start_Second,end_Second):
                print(np_arr[i,k], end=" ")
            print(']')

def draw_grid(img, pxystep=None,major_color=None, pxstep=None,pystep=None):
    #print("{0} XY{1} color{2} X{3} Y{4}".format(img.shape, pxystep,major_color,pxstep,pystep))
    pXYstep = None; pXstep=None; pYstep=None; 
    major_Color=None; minor_Color=None; major_Alpha=None; minor_Alpha=None;
    if pxystep != None:
        pXYstep = pXstep = pYstep = pxystep;
    else:
        pXstep = pxstep if pxstep != None else 100
        pYstep = pystep if pystep != None else 100
    major_Color = major_color if major_color != None else (204, 204, 204) #'#CCCCCC'
    if pXstep != None:
        x = pXstep
        #Draw all lines on X
        while x < img.shape[1]:
            cv2.line(img, (x, 0), (x, img.shape[0]), color=major_Color, thickness=1)
            x += pXstep
    if pYstep != None:
        y = pYstep
        #Draw all lines on Y
        while y < img.shape[0]:
            cv2.line(img, (0, y), (img.shape[1], y), color=major_Color,thickness=1)
            y += pYstep
    return img

def plt_view_image(plt,list_images,figsize=(15,6), axis="off", cmap='gray'):
    plt.figure(figsize=figsize)
    n = len(list_images)  #; print(n)
    plot_number = 1
    for name, img in list_images:
        plt.subplot(1,n,plot_number)
        plt.axis(axis); plt.title(name)
        if cmap =='gray': plt.imshow(img,cmap='gray' )
        else: plt.imshow(img)
        plot_number = plot_number + 1
    plt.show()

def plt_view_grid(plt, axis ='off',
                  xy_sizeaxis =None,
                  xy_measuare =None,
                  x_min=-10, x_max=10, y_min=-10, y_max=10,
                  x_major_size=1, x_minor_size=0.2, y_major_size=1, y_minor_size=0.2,
                  major_color='#CCCCCC', major_alpha=0.5,
                  minor_color='#CCCCCC', minor_alpha=0.2
                 ):
    if xy_sizeaxis is None:  x_min=-10; x_max=10; y_min=-10; y_max=10;
    else: x_min, x_max, y_min, y_max = xy_sizeaxis

    if xy_measuare is None:  x_major_size=1; x_minor_size=0.2; y_major_size=1; y_minor_size=0.2;
    else: x_major_size, x_minor_size, y_major_size, y_minor_size = xy_measuare
        
    plt.xlim(x_min, x_max); plt.ylim(y_min, y_max);
    ax = plt.gca()
    x_major_ticks=np.arange(x_min,x_max,x_major_size); x_minor_ticks=np.arange(x_min,x_max,x_minor_size)
    y_major_ticks=np.arange(y_min,y_max,y_major_size); y_minor_ticks=np.arange(y_min,y_max,y_minor_size)
    ax.set_xticks(x_major_ticks)
    ax.xaxis.set_major_locator(MultipleLocator(x_major_size))
    ax.set_xticks(x_minor_ticks, minor=True)
    ax.set_yticks(y_major_ticks)
    ax.yaxis.set_major_locator(MultipleLocator(y_major_size))
    ax.set_yticks(y_minor_ticks, minor=True)
    plt.grid(which='major', color=major_color, alpha=major_alpha)
    plt.grid(which='minor', color=minor_color, alpha=minor_alpha)
    #plt.gca().invert_yaxis() plt.gca().invert_xaxis()
    
#help("modules")   
import sys             
print('\n'.join(sys.path))
print("current folder ==",os.getcwd())
#pip list

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib
D:\HTML_DOC\Program\opencv\Packt\S09\env
C:\Program Files\Python38\python38.zip
C:\Program Files\Python38\DLLs
C:\Program Files\Python38\lib
C:\Program Files\Python38
d:\html_doc\program\opencv\packt\s09\env

d:\html_doc\program\opencv\packt\s09\env\lib\site-packages
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\pip-20.1-py3.8.egg
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\win32
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\win32\lib
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\Pythonwin
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\IPython\extensions
C:\Users\polit\.ipython
current folder == D:\HTML_DOC\Program\opencv\Packt\S09\env


In [2]:
K = np.array([[560,0,320],[0,560,240],[0,0,1]],dtype=np.float32)
print(K)

[[560.   0. 320.]
 [  0. 560. 240.]
 [  0.   0.   1.]]


In [3]:
rvec = np.array([0.1, 0.2, 0.3], np.float32)
print(rvec)

[0.1 0.2 0.3]


In [4]:
R, _ = cv2.Rodrigues(rvec)
print(R)

[[ 0.9357548  -0.28316498  0.21019171]
 [ 0.30293274  0.9505806  -0.06803132]
 [-0.18054008  0.12733458  0.9752903 ]]


In [5]:
H = K @ R @ np.linalg.inv(K)
print(H)

[[ 8.3258909e-01 -2.1040237e-01  2.1386830e+02]
 [ 2.2555842e-01  1.0051526e+00 -1.1744318e+02]
 [-3.2239300e-04  2.2738318e-04  1.0238841e+00]]


In [6]:
H /= H[2, 2]
print(H)

[[ 8.1316733e-01 -2.0549433e-01  2.0887941e+02]
 [ 2.2029683e-01  9.8170549e-01 -1.1470359e+02]
 [-3.1487257e-04  2.2207903e-04  1.0000000e+00]]


In [7]:
H += np.random.randn(3,3)*0.0001
print(H)

[[ 8.1303173e-01 -2.0548607e-01  2.0887955e+02]
 [ 2.2016945e-01  9.8164731e-01 -1.1470359e+02]
 [-4.4080516e-04  2.7386568e-04  9.9999982e-01]]


In [8]:
np.save('data/rotational_homography.npy', {'H': H, 'K': K})

In [9]:
data = np.load('data/rotational_homography.npy', allow_pickle = True).item()
H, K = data['H'], data['K']


#######################################
print(H)
print(K)

[[ 8.1303173e-01 -2.0548607e-01  2.0887955e+02]
 [ 2.2016945e-01  9.8164731e-01 -1.1470359e+02]
 [-4.4080516e-04  2.7386568e-04  9.9999982e-01]]
[[560.   0. 320.]
 [  0. 560. 240.]
 [  0.   0.   1.]]


In [10]:
H_ = np.linalg.inv(K) @ H @ K
print(H_)

[[ 0.95408934 -0.2931231   0.22114043]
 [ 0.3259627   0.91591954 -0.05459785]
 [-0.2468509   0.15336478  0.9246699 ]]


In [11]:
w, u, vt = cv2.SVDecomp(H_)


print(w, "\n", u, "\n", vt)

[[1.0433289 ]
 [0.97656494]
 [0.94366753]] 
 [[ 0.8833266   0.03747171  0.46725798]
 [ 0.16172671  0.9112331  -0.3788121 ]
 [-0.4399757   0.41018286  0.7988563 ]] 
 [[ 0.96239805 -0.17086798 -0.21117307]
 [ 0.23708142  0.90781486  0.34592575]
 [ 0.13259842 -0.3829835   0.91418886]]


In [12]:
R = u @ vt
print(R)

[[ 0.9209533  -0.29586694  0.2535897 ]
 [ 0.32145205  0.94467586 -0.06523912]
 [-0.22025794  0.14159906  0.9651094 ]]


In [13]:
if cv2.determinant(R) < 0:
    R *= 1
    
print(R)

[[ 0.9209533  -0.29586694  0.2535897 ]
 [ 0.32145205  0.94467586 -0.06523912]
 [-0.22025794  0.14159906  0.9651094 ]]


In [14]:
rvec = cv2.Rodrigues(R)[0]


print('Rotation vector:')
print(rvec)

Rotation vector:
[[0.10643905]
 [0.24384227]
 [0.31767273]]
